### Parsing tickers

In [14]:
f = open("nsdq.html", "r", encoding="utf8")

tickers = []

flag = False
for i, line in enumerate(f):
    if 'class="stock__ticker nobr" target="_self">' in line:
        flag = True
        continue
    if flag:
        flag = False
        tickers.append(line.strip())

In [18]:
tickers[0], tickers[-1]

('AAPL', 'ZM')

##### II

In [110]:
tickers = pd.read_csv("nasdaq.csv", sep=";").drop(columns=["Id"])["Symbol"].to_list()

In [111]:
tickers[0], tickers[-1]

('MSFT', 'RIVN')

### Downloading the data

In [21]:
import yfinance as yf

In [112]:
period = "2y"
interval='1d'

In [113]:
data = yf.download(tickers, period=period, interval=interval)["Close"]

[*********************100%***********************]  101 of 101 completed


In [114]:
data

,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,...,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-24,127.099998,135.910004,497.829987,162.509995,197.039993,287.480011,86.570000,604.450012,134.580002,77.440002,...,202.146667,188.690002,174.169998,211.220001,54.830002,31.280001,234.000000,71.669998,321.559998,174.399994
2021-05-25,126.900002,133.990005,505.079987,163.720001,195.199997,289.470001,85.930000,594.950012,137.500000,77.860001,...,201.563339,188.699997,173.699997,208.759995,54.189999,30.660000,236.570007,71.239998,321.279999,172.740005
2021-05-26,126.849998,134.750000,506.980011,163.880005,194.380005,292.510010,85.750000,596.690002,136.899994,78.339996,...,206.376663,188.360001,172.979996,206.800003,52.009998,31.820000,237.070007,71.000000,326.049988,194.199997
2021-05-27,125.279999,143.169998,498.279999,163.210007,197.240005,286.670013,85.550003,599.770020,137.820007,78.419998,...,210.283340,189.130005,172.130005,207.309998,53.040001,32.520000,229.350006,70.269997,326.429993,194.259995
2021-05-28,124.610001,140.399994,504.579987,164.600006,196.020004,285.859985,86.000000,590.150024,138.130005,80.080002,...,208.406662,189.820007,172.830002,208.630005,52.660000,32.110001,228.720001,70.879997,331.529999,194.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-18,175.050003,111.870003,360.429993,192.259995,215.429993,204.839996,86.639999,293.420013,129.919998,107.930000,...,176.889999,169.809998,220.630005,339.459991,31.459999,12.400000,195.750000,65.870003,69.529999,123.959999
2023-05-19,175.160004,107.379997,371.250000,190.529999,216.179993,200.889999,86.559998,290.989990,126.949997,105.820000,...,180.139999,170.110001,222.119995,341.700012,31.280001,12.200000,195.710007,65.930000,69.370003,125.269997
2023-05-22,174.199997,109.930000,372.049988,191.309998,216.529999,203.490005,86.410004,300.829987,126.550003,108.000000,...,188.869995,170.860001,218.250000,339.970001,31.190001,12.160000,196.410004,65.879997,71.410004,128.860001


### Modeling

In [95]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.clique import find_cliques

In [115]:
threshold = 0.5
corr_arr = data.corr().to_numpy()[np.triu_indices(len(tickers), k = 1)]
print(f"Min: {min(corr_arr)}, Max: {max(corr_arr)}")

corr_data = data.corr()
corr_data[corr_data < threshold] = 0
corr_data[corr_data >= threshold] = 1
corr_data = corr_data.astype(int)
corr_data

Min: -0.9336489038780564, Max: 0.9983226164605881


,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,...,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL,ZM,ZS
AAPL,1,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
ABNB,0,1,1,0,0,1,0,1,1,1,...,1,1,1,0,1,1,1,0,1,1
ADBE,0,1,1,0,0,1,0,1,1,1,...,1,1,1,0,1,1,1,0,1,1
ADI,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
ADP,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBD,0,1,1,0,0,1,0,1,1,1,...,0,1,0,0,1,1,1,0,1,1
WDAY,0,1,1,0,0,1,0,1,1,1,...,1,1,1,0,1,1,1,0,1,1
XEL,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
ZM,0,1,1,0,0,1,0,1,1,0,...,0,1,0,0,1,1,1,0,1,1


In [116]:
G = nx.Graph(corr_data, nodetype=int)

In [117]:
cliques = [(len(clique), clique) for clique in find_cliques(G)]
cliques.sort(reverse=True)
clique_num = cliques[0][0]
max_clique_intersection = set(cliques[0][1])
for len_clique in cliques:
    if len_clique[0] == clique_num:
        max_clique_intersection &= set(len_clique[1])
max_clique_intersection

{'ABNB',
 'ADBE',
 'ADSK',
 'ALGN',
 'AMAT',
 'AMZN',
 'ANSS',
 'ASML',
 'CHTR',
 'CMCSA',
 'CSCO',
 'DDOG',
 'EBAY',
 'GOOG',
 'GOOGL',
 'IDXX',
 'ILMN',
 'INTC',
 'INTU',
 'ISRG',
 'LCID',
 'LRCX',
 'META',
 'MRVL',
 'MU',
 'NFLX',
 'NXPI',
 'RIVN',
 'TEAM',
 'TXN',
 'WBA',
 'WBD',
 'WDAY',
 'ZS'}

In [118]:
def get_max_cliq_inter(data, threshold):
    corr_data = data.corr()
    corr_data[corr_data < threshold] = 0
    corr_data[corr_data >= threshold] = 1
    corr_data = corr_data.astype(int)
    G = nx.Graph(corr_data, nodetype=int)
    cliques = [(len(clique), clique) for clique in find_cliques(G)]
    cliques.sort(reverse=True)
    clique_num = cliques[0][0]
    max_clique_intersection = set(cliques[0][1])
    max_clique_count = 0
    for len_clique in cliques:
        if len_clique[0] == clique_num:
            max_clique_intersection &= set(len_clique[1])
            max_clique_count += 1
    return max_clique_intersection, max_clique_count

In [142]:
thresholds = [i / 100 for i in range(50, 91, 5)]
thresholds.append(0.92)
todata = []
for threshold in thresholds:
    inter_clique, max_cli_count = get_max_cliq_inter(data, threshold)
    todata.append((threshold, max_cli_count, list(inter_clique) ))
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.DataFrame(todata, columns=["Threshold", "Max clique count", "Clique intersection"])

,Threshold,Max clique count,Clique intersection
0,0.50,4,"[RIVN, LCID, WBA, CSCO, MU, AMZN, ILMN, ADBE, TXN, MRVL, ISRG, IDXX, CHTR, EBAY, TEAM, ANSS, ASML, ALGN, DDOG, META, GOOG, WDAY, INTC, LRCX, ADSK, NFLX, WBD, ZS, ABNB, NXPI, CMCSA, GOOGL, AMAT, INTU]"
1,0.55,2,"[RIVN, WBA, CSCO, CRWD, MU, AMZN, ILMN, ADBE, ISRG, IDXX, CHTR, EBAY, TEAM, ANSS, ASML, ALGN, META, GOOG, WDAY, INTC, LRCX, ADSK, NFLX, WBD, ZS, ABNB, JD, NXPI, CMCSA, GOOGL, AMAT, INTU]"
2,0.60,3,"[WBA, CSCO, AMZN, ILMN, ADBE, ISRG, IDXX, CHTR, EBAY, TEAM, ANSS, ASML, ALGN, META, GOOG, WDAY, INTC, LRCX, ADSK, NFLX, WBD, ZS, ABNB, NXPI, CMCSA, GOOGL, AMAT, INTU]"
3,0.65,4,"[CSCO, AMZN, ILMN, ADBE, PYPL, ISRG, IDXX, CHTR, EBAY, ANSS, ALGN, META, GOOG, WDAY, INTC, ADSK, NFLX, WBD, ZS, ABNB, NXPI, CMCSA, GOOGL, AMAT, INTU]"
4,0.70,3,"[CSCO, AMZN, ILMN, ADBE, IDXX, CHTR, EBAY, ANSS, ALGN, META, GOOG, WDAY, INTC, ADSK, NFLX, WBD, ABNB, NXPI, CMCSA, GOOGL, INTU]"
5,0.75,21,"[META, NXPI, WDAY, EBAY, CMCSA, ILMN, ALGN, ADBE]"
6,0.80,5,"[META, AMAT, WDAY, LRCX, EBAY, ASML, ANSS, NFLX, ALGN, ADBE]"
7,0.85,1,"[META, CHTR, EBAY, CMCSA, ALGN, ADSK, ZM, ILMN, ADBE, PYPL]"
8,0.90,4,"[EBAY, ALGN, ILMN, CMCSA]"
9,0.92,3,"[EBAY, ALGN, CMCSA]"


In [130]:
fdf = pd.read_csv("nasdaq.csv", sep=";").drop(columns=["Id"])
fdf[fdf["Symbol"].isin(['EBAY', 'ALGN', 'CMCSA'])]

,Company,Symbol,Weight,Price,Chg,%Chg
12,Comcast Corp,CMCSA,1.262,41.08,-0.17,(-0.40%)
91,eBay Inc,EBAY,0.170,43.61,-0.51,(-1.17%)
92,Align Technology Inc,ALGN,0.162,290.26,-10.60,(-3.51%)
